# (S1) Initialization


### Load dependency files

In [2]:
import tensorflow as tf
import datetime
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

print("Tensorflow version={}".format(tf.__version__))
print("Successfully loaded files. {}".format(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')))

/Users/thchang/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensorflow version=1.8.0-dev20180328
Successfully loaded files. 2018-05-14 22:31:59


### References

In [64]:
# Mikolov paper on word2Vec: https://arxiv.org/pdf/1301.3781.pdf
# Tensorflow embedding instruction: https://github.com/tensorflow/models/tree/master/tutorials/embedding
# Refernce local code on calculate word2vec embedding: /Users/thchang/Documents/dev/git/tensorlfow/models/tutorials/embedding

### Helper functions

In [2]:
def getTime():
    return datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    
def initVariables():
    init = tf.global_variables_initializer()
    init.run()

# Used for tensorboard
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
    
print("Successfully defined helper functions. {}".format(getTime()))

Successfully defined helper functions. 2018-04-26 21:41:59


### Nodes

In [ ]:
# cd /Users/thchang/Documents/dev/git/tensorlfow/models/tutorials/embedding
# python word2vec_optimized.py --train_data=data/text8 --eval_data=data/questions-words.txt --save_path=/tmp/

# (S2) Tensorflow Training Code

### Plan

In [65]:
"""
(1) cd /Users/thchang/Documents/dev/git/tensorlfow/models/tutorials/embedding
(2) To execute: (per instruction in  https://github.com/tensorflow/models/tree/master/tutorials/embedding)
    python word2vec_optimized.py --train_data=text8 --eval_data=questions-words.txt --save_path=/tmp/

Action Items
    - Update text8 with my queryAsect test fie
    - Create my own question-words.txt
        Similar queries are queries that share N aspects
        Calculate cosine simlarity of similar items, and make sure it's less than 
    - How to feed multi-term queries to predict n aspects ...
        
    
Done
    - Look at how eval data is used in the code: PyCharm: word2vec_optimized.py
        eval data is used for evaluation for the test data
"""

'\n'

# (S3) Tensorflow Review/Appendix

### (A) Tensor Flow Program Structure
Cumbersome, but this give TF the ability for parallel execution across multiple platforms

#### -  Defintion of Computation Graph

* Nodes = [ Tensor :{constants, variables, placeholders}, Operations:{} ]
* Edge = Tensor that results from the operations

#### - Execution
* Session objects encapsulte the environment tensor and operations objects are evaluated.


### (A.1) Tensor: N dimensional Matrix


#### - Tensor: Constants: 
* Value cannot be changed
* Stored as part of the computation graph, and is memory expensive.

In [9]:
# Constant - 0 dimension
tensorConstant1D = tf.constant(0.1)

# Vector
tensorVector = tf.constant(0.1, shape=[100])

# 2 dimensional matrix
tensor2DMatrixZero = tf.zeros([784, 10], tf.int32)
tensor2DMatrixNormal = tf.truncated_normal([784, 10], stddev=0.1)

# Tensorflow types: tf.float32, tf.float64, tf.int8, tf.uint8, tf.string, tf.bool, tf.complex64, tf.qint32

#### - Tensor: Variables
* Variable are required when values requires updating WITHIN a session. 
* Needs to be initalized before use.
* Contants are stored in the computation graph definition, ie very memory expansive. In comparision, variables are stored separately, and exist on the parameter server
* Example: weights and biases of neural network

In [5]:
# Use case 1: Simple
varX = tf.Variable(3, name='x')
varY = tf.Variable(4, name='y')

# Use case 2: Neural network
varW = tf.Variable(tf.zeros([784, 10])) #initial weights of our neural network
varB = tf.Variable(tf.zeros([10])) # Bias of our neural network

# Use case 3: More complicated initilizations
image_size=540
hidden_size = 10
varWInput = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_size], stddev=0.1))
varBInput = tf.Variable(tf.constant(0.1, shape=[hidden_size]))
varWInput2 = tf.Variable(tf.random_uniform([50,50], minval=0, maxval=10, seed=0))

#### - Tensor: PlaceHolders: 
* Objects whose value that are fed INTO the computation graph, at each training batch
* PH are used to feed data into the graph via sess.run().
* Example: input vectors and labels of our neural network

In [6]:
# Use case 1: Simple
phX = tf.placeholder(tf.int64, name = 'x')

# Use case 2: Neural network
phInputVectorX = tf.placeholder(tf.float32, shape=[None, 784]) #input to our neural network model
phLabelY = tf.placeholder(tf.float32, shape=[None, 10])

In [27]:
# Use case 2:
xPh = tf.placeholder(tf.float32)
yOperation = 2 * xPh
dataVar = tf.random_uniform([4,5], maxval=10, dtype=tf.float32)
with tf.Session() as sess:
    x_result = sess.run(dataVar)
    print("x_result:\n{}".format(x_result))
    
    y_result = sess.run(yOperation, feed_dict = {xPh: x_result})
    print("y_result:\n{}".format(y_result))

x_result:
[[0.9714329 7.329994  6.544899  4.8730717 1.3280773]
 [6.366893  1.7521656 9.374327  1.175015  3.7473881]
 [6.345556  3.9288676 9.876809  8.137418  4.8428845]
 [5.817404  5.1352034 2.5477123 0.471375  6.135892 ]]
y_result:
[[ 1.9428658 14.659988  13.089798   9.746143   2.6561546]
 [12.733786   3.504331  18.748653   2.35003    7.4947762]
 [12.691112   7.857735  19.753618  16.274836   9.685769 ]
 [11.634808  10.270407   5.0954247  0.94275   12.271784 ]]


### (A.2) Operation

In [7]:
# Use case 1: Simple- element wise operatrions
fOperation = varX*varX*varY + varY + 2

# Use case 2: Neural network - matrix operations: tf.matmul, 
fOperationNN = tf.matmul(phInputVectorX, varW) + varB

### (A.3) Session: Needed to evaluate a graph

In [8]:
def showVariableFlow():
    with tf.Session() as sess: # Automatically closes the session
        # Define and initialize variables
        varX = tf.Variable(3, name='x')
        varY = tf.Variable(4, name='y')
        initVariables()

        # Define operation. Var + Operation = computation graph
        fOperation = varX*varX*varY + varY + 2

        # Uses the session in this scope to execute the graph
        resultOfOperation = fOperation.eval()
        
        print("showVariableFlow(): resultOfOperation={}".format(resultOfOperation))
    
def showPlaceHolderFlow():
    with tf.Session() as sess:
        phX = tf.placeholder(tf.int64, name = 'x')
        fOperation = phX + 5        
        resultOfOperation = fOperation.eval(feed_dict={phX: [[1, 2, 3]]})
        print("showPlaceHolderFlow(): resultOfOperation={}".format(resultOfOperation))
        
def showMNISTFlow():
    from tensorflow.examples.tutorials.mnist import input_data

    def train_network(training_data, labels, output, keep_prob=tf.placeholder(tf.float32)):
        learning_rate = 1e-4
        steps_number = 1000
        batch_size = 100

        # Read data
        mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

        # Define the loss function
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=output))

        # Define optimizers: 
        # Adam = Adaptive learning rate using first and second moments of gradients...
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

        # Accuracy calculation
        correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #Computes the mean of elements across dimensions of a tensor

        # Run the training
        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())

        for i in range(steps_number):
            # Get the next batch
            input_batch, labels_batch = mnist.train.next_batch(batch_size)

            # Print the accuracy progress on the batch every 100 steps
            if i%100 == 0:
                train_accuracy = accuracy.eval(feed_dict={training_data: input_batch, labels: labels_batch, keep_prob: 1.0})
                print("Step %d, training batch accuracy %g %%"%(i, train_accuracy*100))

            # Run the training step
            train_step.run(feed_dict={training_data: input_batch, labels: labels_batch, keep_prob: 0.5})
            
            
        print("The end of training!")

        # Evaluate on the test set
        test_accuracy = accuracy.eval(feed_dict={training_data: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0})
        print("Test accuracy: %g %%"%(test_accuracy*100))
        
    image_size = 28
    labels_size = 10
    hidden_size = 1024

    # Define placeholders, used for input into simulation.
    # Placeholders are assiged values via feed_dict, like this: train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    training_data = tf.placeholder(tf.float32, [None, image_size*image_size])
    labels = tf.placeholder(tf.float32, [None, labels_size])

    # Variables for the hidden layer
    W_input = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_size], stddev=0.1))
    b_input = tf.Variable(tf.constant(0.1, shape=[hidden_size]))
    
    # Hidden layer with reLU activation function
    y_input = tf.nn.relu(tf.matmul(training_data, W_input) + b_input)

    # Variables for the output layer
    W = tf.Variable(tf.truncated_normal([hidden_size, labels_size], stddev=0.1))
    b = tf.Variable(tf.constant(0.1, shape=[labels_size]))
    
    # Connect hidden to the output layer
    output = tf.matmul(y_input, W) + b
    
    # Train & test the network
    train_network(training_data, labels, output)
    

#showVariableFlow()
#showPlaceHolderFlow()
showMNISTFlow()
#showMNISTFlowTensorBoard()

NameError: name 'showMNISTFlowTensorBoard' is not defined

### (B) Tensor Board


In [48]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

logDir = "/Users/thchang/TensorBoard/logs"
max_steps=500
fake_data=False
dropout=0.9

def train():
    
  # Import data
  mnist = input_data.read_data_sets("/tmp/tensorflow/mnist/logs/mnist_with_summaries", fake_data=False)

  sess = tf.InteractiveSession()
  # Create a multilayer model.

  # Input placeholders
  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.int64, [None], name='y-input')

  with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

  # We can't initialize these variables to 0 - the network will get stuck.
  def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

  def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

  def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)

  def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses ReLU to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights)
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases)
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.summary.histogram('pre_activations', preactivate)
      activations = act(preactivate, name='activation')
      tf.summary.histogram('activations', activations)
      return activations

  hidden1 = nn_layer(x, 784, 500, 'layer1')

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

  # Do not apply softmax activation yet, see below.
  y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

  with tf.name_scope('cross_entropy'):
    # The raw formulation of cross-entropy,
    #
    # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
    #                               reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.losses.sparse_softmax_cross_entropy on the
    # raw logit outputs of the nn_layer above, and then average across
    # the batch.
    with tf.name_scope('total'):
      cross_entropy = tf.losses.sparse_softmax_cross_entropy(
          labels=y_, logits=y)
  tf.summary.scalar('cross_entropy', cross_entropy)

  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(
        cross_entropy)

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_parediction = tf.equal(tf.argmax(y, 1), y_)
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', accuracy)

  # Merge all the summaries and write them out to
  # /tmp/tensorflow/mnist/logs/mnist_with_summaries (by default)
  merged = tf.summary.merge_all()
  train_writer = tf.summary.FileWriter(logDir + '/train', sess.graph)
  test_writer = tf.summary.FileWriter(logDir + '/test')
  tf.global_variables_initializer().run()

  # Train the model, and also write summaries.
  # Every 10th step, measure test-set accuracy, and write test summaries
  # All other steps, run train_step on training data, & add training summaries

  def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    """
    if train or fake_data:
      xs, ys = mnist.train.next_batch(100, fake_data=fake_data)
      k = dropout
    else:
      xs, ys = mnist.test.images, mnist.test.labels
      k = 1.0
    """
  xs, ys = mnist.train.next_batch(100, fake_data=fake_data) 
  k = dropout
  return {x: xs, y_: ys, keep_prob: k}

  for i in range(max_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
      if i % 100 == 99:  # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        train_writer.add_summary(summary, i)
        print('Adding run metadata for', i)
      else:  # Record a summary
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
  train_writer.close()
  test_writer.close()

def main(_):
  #if tf.gfile.Exists(logDir):
  #  tf.gfile.DeleteRecursively(FLAGS.logDir)
  #tf.gfile.MakeDirs(logDir)
  train()
    
if __name__ == '__main__':
     tf.app.run(main=main,  argv=[sys.argv[0]])


Extracting /tmp/tensorflow/mnist/logs/mnist_with_summaries/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/logs/mnist_with_summaries/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/logs/mnist_with_summaries/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/logs/mnist_with_summaries/t10k-labels-idx1-ubyte.gz


SystemExit: 